# Synthetic Indoor Scene Dataset

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

In [21]:
ex = '/ssd1/softgroup/preprocess/Area_5_hallway_7_inst_nostuff.pth'

In [22]:
data = torch.load(ex)

In [23]:
data

(array([[-7.001,  5.411,  3.053],
        [-7.038,  5.393,  3.053],
        [-6.986,  5.435,  3.053],
        ...,
        [-6.359,  5.36 ,  2.33 ],
        [-6.36 ,  5.36 ,  2.333],
        [-6.356,  5.362,  2.335]], dtype=float32),
 array([[ 0.00392157,  0.00392157, -0.01176471],
        [-0.25490196, -0.21568627, -0.24705882],
        [-0.01176471, -0.00392157, -0.04313725],
        ...,
        [ 0.38823529,  0.42745098,  0.45882353],
        [ 0.38039216,  0.41960784,  0.45098039],
        [ 0.38039216,  0.41960784,  0.45098039]]),
 array([0, 0, 0, ..., 2, 2, 2]),
 array([13, 13, 13, ..., 20, 20, 20]),
 array([2]),
 'Area_5_hallway_7')

In [24]:
xyz = data[0]
rgb = data[1]
semantic_label = data[2]
instance_label = data[3]

In [26]:
np.unique(instance_label)

array([-100,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29])

In [2]:
path = '/ssd1/synthetic_rooms/'
save_dir = '/ssd1/synthetic_inst_preprocessed'

In [27]:
mode = 'train'

In [28]:
stat = {
    -1: {'total': 0, 'num': 0},
    0: {'total': 0, 'num': 0},
    1: {'total': 0, 'num': 0},
    2: {'total': 0, 'num': 0},
    3: {'total': 0, 'num': 0},
    4: {'total': 0, 'num': 0},
}

In [30]:
for subdir in os.listdir(path):
    print(subdir)
    split = f'{path}/{subdir}/{mode}.lst'
    files = []
    with open(split, 'r') as f:
        files = f.read().split('\n')
    for file in files:
        num = np.random.randint(10)
        p = f'{path}{subdir}/{file}/pointcloud/pointcloud_0{num}.npz'
        try:
            data = np.load(p, allow_pickle=True)
            xyz = data['points'].astype('float32')
            instance_labels = data['semantics']
            semantic_labels = data['semantics']
            pp = 0
            for k in np.unique(semantic_labels):
                stat[k]['total'] = len(xyz[semantic_labels==k])
                stat[k]['num'] += 1
                instance_labels[semantic_labels==k] = pp
                pp += 1
            print("sem", np.unique(semantic_labels), "inst", np.unique(instance_labels))
            rgb = np.zeros((xyz.shape[0], 3)).astype('uint8')
            room_label = file
            scene = f'{subdir}_{file}'
            save_path = f'{save_dir}/{mode}/{scene}_inst_nostuff.pth'
            torch.save((xyz, rgb, semantic_labels, instance_labels, room_label, scene), save_path)
        except:
            continue
        
    print(stat)

        

rooms_05
sem [-1  0  1  2  3  4] inst [0 1 2 3 4 5]
{-1: {'total': 23314, 'num': 15}, 0: {'total': 8035, 'num': 11}, 1: {'total': 12667, 'num': 13}, 2: {'total': 6796, 'num': 12}, 3: {'total': 10021, 'num': 11}, 4: {'total': 13552, 'num': 10}}
rooms_04
sem [-1  0  1  2  4] inst [0 1 2 3 4]
sem [-1  0  1  2  3] inst [0 1 2 3 4]
sem [-1  1  2  4] inst [0 1 2 3]
sem [-1  0  2  3  4] inst [0 1 2 3 4]
sem [-1  1  2  3] inst [0 1 2 3]
sem [-1  1  2  3  4] inst [0 1 2 3 4]
sem [-1  1  2  3  4] inst [0 1 2 3 4]
sem [-1  0  1  3] inst [0 1 2 3]
sem [-1  0  1  3  4] inst [0 1 2 3 4]
sem [-1  0  3  4] inst [0 1 2 3]
sem [-1  0  1  2  3] inst [0 1 2 3 4]
sem [-1  0  1  2  4] inst [0 1 2 3 4]
sem [-1  0  1  2] inst [0 1 2 3]
{-1: {'total': 23054, 'num': 28}, 0: {'total': 13042, 'num': 20}, 1: {'total': 24207, 'num': 24}, 2: {'total': 17711, 'num': 22}, 3: {'total': 7242, 'num': 20}, 4: {'total': 12377, 'num': 18}}


In [45]:
np.unique(semantic_labels)

array([-1,  0,  1,  4])